In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
#btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
#btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

In [16]:
vol0 = get_single_volume(btf_fname_red, 0, num_slices=33, alpha=0.15)

In [17]:
alpha = 0.15
dat_foldername = r'..\point_cloud_alignment'
fname = os.path.join(dat_foldername, 'img100.tif')
f = lambda tif : (alpha*tif.asarray()).astype('uint8')
with tifffile.TiffFile(fname) as tif0:
    vol0_old = f(tif0)

## Use single function to track

In [20]:
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *


In [51]:
all_matches, all_conf, all_neurons = track_neurons_full_video(btf_fname_red,
                             start_frame=0,
                             num_frames=3,
                             num_slices=33,
                             alpha=0.15,
                             verbose=2)

Matching frames 0 and 1 (end at 3)
Matching frames 1 and 2 (end at 3)
Finished 3 frames in 30.148225784301758 seconds


In [40]:
clust_df = build_tracklets_from_matches(all_neurons, all_matches, verbose=2)

0 / 2


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [43]:
n = all_neurons[0]

In [46]:
n2 = np.array([r for r in n])

In [47]:
n2


array([[  6.        , 405.47446289, 269.86676636],
       [  7.5       , 425.46964264, 240.61898994],
       [  7.5       , 417.27278137, 247.66906166],
       [  7.        , 401.239951  , 227.98054722],
       [  6.5       , 372.39982096, 301.49717712],
       [  7.5       , 419.25033569, 277.7630806 ],
       [  7.5       , 409.27627055, 241.19137319],
       [  8.        , 367.72521537, 319.98017229],
       [ 10.5       , 373.39053955, 344.57333069],
       [  7.5       , 416.37030792, 228.77507401],
       [  9.5       , 400.12728119, 215.41918564],
       [  9.5       , 460.54410553, 198.54844666],
       [  9.5       , 480.78538513, 213.92084503],
       [ 12.5       , 434.4360466 , 250.36151123],
       [ 12.        , 375.8926566 , 236.89836121],
       [ 12.5       , 417.9247818 , 265.31040192],
       [ 13.        , 494.81366403, 202.02605329],
       [ 13.        , 430.84035645, 215.67511902],
       [ 13.5       , 372.49023946, 372.02395121],
       [ 13.        , 480.56080

# Aside: timing tests

In [15]:
# Open video
def open_tif(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        1+1

def open_page(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat = page.asarray()
            if i>0:
                break

def open_volume(vid_fname, num_slices=33):
    dat = []
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat.append(page.asarray())
            if i>num_slices:
                break
                
def open_with_imread(vid_fname, num_slices=33):
    dat = tifffile.imread(vid_fname, key=range(num_slices))

In [9]:
%timeit open_tif(btf_fname_red)

269 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit open_page(btf_fname_red)

2 ms ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit open_volume(btf_fname_red)

38.8 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit open_with_imread(btf_fname_red)

31.5 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
